# MAIA Demo

#### Many of MAIA's experiments are available in the [experiment browser](https://multimodal-interpretability.csail.mit.edu/maia/experiment-browser/) ####

In [3]:
import os

import openai
from dotenv import load_dotenv

# Some imports require api key to be set ######
# Load environment variables
load_dotenv()

# Load OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.organization = os.getenv("OPENAI_ORGANIZATION")
print(type(openai.api_key))
###############################################


from maia_api import System, Tools
from utils.DatasetExemplars import DatasetExemplars
from utils.main_utils import generate_save_path
from utils.InterpAgent import InterpAgent

<class 'str'>


In [4]:
# Login for access to sd-3.5
from huggingface_hub import login
login()

In [16]:
maia_model = 'gpt-4o-mini'
task = 'neuron_description'
n_exemplars = 15
model_name = "resnet152"
layer = "layer4"
neuron_num = 20
images_per_prompt = 1
path2save = '../results'
path2prompts = './prompts'
path2exemplars = './exemplars'
device = 1
text2image = 'sd'
debug = True

unit_config = {model_name: {layer: [neuron_num]}}

path2save = generate_save_path(path2save, maia_model, "test")
print(path2save)
os.makedirs(path2save, exist_ok=True)

../results/gpt-4o-mini/test


In [17]:
maia = InterpAgent(
            model_name=maia_model,
            prompt_path=path2prompts,
            api_prompt_name="api.txt",
            user_prompt_name=f"user_{task}.txt",
            overload_prompt_name="final.txt",
            end_experiment_token="[FINAL]",
            max_round_count=25,
            debug=debug
        )
net_dissect = DatasetExemplars(path2exemplars, n_exemplars, path2save, unit_config)
system = System(model_name, layer, neuron_num, net_dissect.thresholds, device)
tools = Tools(path2save, device, maia, system, net_dissect, images_per_prompt=images_per_prompt, text2image_model_name=text2image, image2text_model_name=maia)

print(tools.html_path)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

../results/gpt-4o-mini/test/experiment_0.html


In [ ]:
maia.run_experiment(system, tools, save_html=True)